In [47]:
from binance.client import Client
import config
import pandas as pd

In [48]:
exchange = Client(api_key=config.key,api_secret=config.secret,tld='us',testnet=True)

In [49]:
def getData(symbol,time,client):
    """
    1504541580000, // UTC timestamp in milliseconds, integer
    4235.4,        // (O)pen price, float
    4240.6,        // (H)ighest price, float
    4230.0,        // (L)owest price, float
    4230.7,        // (C)losing price, float
    37.72941911    // (V)olume float (usually in terms of the base currency, the exchanges docstring may list whether quote or base units are used)
    """
    klines = client.get_historical_klines(symbol=symbol, interval=time)

    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(klines, columns=columns)

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC')
    df['timestamp'] = df['timestamp'].dt.tz_convert('America/Chicago')

    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    # df.set_index('Date', inplace=True)
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    return df

In [50]:
def calculate_macd(data, **params):
    """
    Calculates the MACD line, signal line, and zero line for a given DataFrame of stock data.
    :param data: DataFrame containing stock data with at least a 'Close' column and a DatetimeIndex.
    :param EMA Long Period: The period length for the long EMA. Default is 26.
    :param EMA Short Period: The period length for the short EMA. Default is 12.
    :param Signal Line Period: The period length for the signal line. Default is 9.
    :return: DataFrame containing the MACD line, signal line, and zero line as columns, with the same index as the input DataFrame.
    """
    ema_long = data['Close'].ewm(span=params['EMA Long Period'], adjust=False).mean()
    ema_short = data['Close'].ewm(span=params['EMA Short Period'], adjust=False).mean()
    macd_line = ema_short - ema_long
    signal_line = macd_line.ewm(span=params['Signal Line Period'], adjust=False).mean()
    zero_line = pd.Series(0, index=data.index)
    macd_df = pd.concat([macd_line, signal_line, zero_line], axis=1)
    macd_df.columns = ['MACD_line', 'Signal_line', 'Zero_line']
    return macd_df


In [51]:
class MyStrategy:
    
    def generate_signals_backtest(self,data,user_input,**params):
       
        # data['Signal'] = data['Signal'].astype("float")
        if user_input == 'MACD':
            signals = self.MACD_strategy(data,**params)
        elif user_input == 'RSI':
            signals = self.RSI_strategy(data,**params)
        return signals 
    
    def RSI_strategy(self,data,**params):
        from ta.momentum import rsi
        
        # Define your trading signals here
        lower_limit= params['Buy Threshold'] 
        upper_limit = params['Sell Threshold']
        
        # Calculate the RSI indicator
        #data["RSI"] = rsi(data["Close"],window=params['RSI Period'])
        data['RSI']= ta.momentum.RSIIndicator(data["Close"], window=params['RSI Period']).rsi()
       
        #data = rsi_calculation(data,params['RSI Period'])

        #data["RSI"] = ta.momentum.RSIIndicator(close=data["close"], window=params['RSI Period']).rsi()
        data = data.dropna()
        data = data.reset_index(drop=True)
        # # Generate the signals based on the RSI value
        #data["signal"] = data['RSI'].apply(lambda x: 1 if x < lower_limit else -1 if x > upper_limit else 0)
        # Create the 'signal' column
        #data = data.copy()
        data['Signal'] = 0
        
        
        # Set a flag to track if we're currently in a position
        in_position = False
        
        # Loop through each row and set the signal based on the RSI and previous position
      
        for i in range(1, len(data)):
            rsi_value = data['RSI'][i]
           
            if in_position:
                # If we're currently in a position, continue holding until a Sell signal
                if rsi_value >= upper_limit:
                    data['Signal'][i] = -1
                    in_position = False
                else:
                    data['Signal'][i] = 0
            else:
                # If we're not currently in a position, Buy if RSI is below the lower limit
                if rsi_value <= lower_limit:
                    data['Signal'][i] = 1
                    in_position = True
                else:
                    data['Signal'][i] = 0
        return data
    def MACD_strategy(self,data,**params):
        #MACD trading strategy
        macd_df = calculate_macd(data, **params)
        data = pd.concat([data, macd_df], axis=1)
        # Set a flag to track if we're currently in a position
        in_position = False
        
        # Loop through each row and set the signal based on the RSI and previous position
        data['Signal'] = 0
        for i in range(1, len(data)):
            macd = data['MACD_line'][i]
            zero_line = data['Zero_line'][i]
            signal_line = data['Signal_line'][i]
           
            if in_position:
                # If we're currently in a position, continue holding until a Sell signal
                if (macd < signal_line) & (macd > zero_line) & (signal_line > zero_line): 
                    data['Signal'][i] = -1
                    in_position = False
                else:
                    data['Signal'][i] = 0
            else:
                # If we're not currently in a position, Buy if RSI is below the lower limit
                if (macd > signal_line) & (macd < zero_line) & (signal_line < zero_line):
                    data['Signal'][i] = 1
                    in_position = True
                else:
                    data['Signal'][i] = 0

        return data

In [52]:
 # #MACD
symbol = "ETHUSDT"
time_frame = '15m'
df = getData(symbol,time_frame,exchange) 
strategy = MyStrategy()

In [53]:
inital_parameters = {'EMA Long Period': 11.0, 'EMA Short Period': 6.0, 'Signal Line Period': 14.0}

In [54]:
signals = strategy.generate_signals_backtest(df,'MACD',**inital_parameters)
signals

C:\Users\jdlugosz\AppData\Local\Temp\ipykernel_4660\1502439823.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'][i] = 0
C:\Users\jdlugosz\AppData\Local\Temp\ipykernel_4660\1502439823.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'][i] = 1
C:\Users\jdlugosz\AppData\Local\Temp\ipykernel_4660\1502439823.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'][i] = 0
C:\Users\jdlugosz\AppData

,Date,Open,High,Low,Close,Volume,MACD_line,Signal_line,Zero_line,Signal
0,2023-03-19 04:15:00-05:00,1773.30,1774.19,1769.11,1770.55,117.91569,0.000000,0.000000,0,0
1,2023-03-19 04:30:00-05:00,1770.01,1777.65,1770.01,1777.33,169.10040,0.807143,0.107619,0,0
2,2023-03-19 04:45:00-05:00,1776.94,1779.03,1775.92,1777.19,87.47843,1.232483,0.257601,0,0
3,2023-03-19 05:00:00-05:00,1777.29,1777.44,1774.05,1774.05,91.76275,1.053162,0.363676,0,0
4,2023-03-19 05:15:00-05:00,1773.79,1779.42,1772.35,1778.10,138.48341,1.378416,0.498975,0,0
...,...,...,...,...,...,...,...,...,...,...
995,2023-03-29 13:00:00-05:00,1798.51,2000.00,1796.07,1799.39,91.61293,-1.104015,-1.147507,0,0
996,2023-03-29 13:15:00-05:00,1799.25,1801.87,1795.90,1799.89,97.91090,-0.843816,-1.107015,0,0
997,2023-03-29 13:30:00-05:00,1800.09,1805.91,1799.62,1803.25,96.48204,-0.248754,-0.992580,0,0
998,2023-03-29 13:45:00-05:00,1803.23,1805.80,1801.49,1804.26,114.94666,0.237533,-0.828565,0,0


In [55]:
# df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close','Signal'])

# # Populate the DataFrame with your data
# df['Date'] = ['2022-01-01', '2022-01-02', '2022-01-03','2022-01,04']
# df['Open'] = [100.0, 101.0, 99.0,100.0]
# df['High'] = [102.0, 103.0, 100.0,105.0]
# df['Low'] = [99.0, 99.5, 98.0,97.8]
# df['Close'] = [101.0, 102.5, 98.5,103.5]

# df['Signal'] = [1, 0, -1,0]

In [56]:
import backtrader as bt
import matplotlib
%matplotlib inline
matplotlib.pyplot.ion()

In [57]:
class MyStrategy(bt.Strategy):
    def __init__(self):
        self.signal = self.data.params.signal

    def next(self):
        if self.signal[0] == 1:
            self.buy()
        elif self.signal[0] == -1:
            self.sell()



In [58]:
signals.dtypes

Date           datetime64[ns, America/Chicago]
Open                                   float64
High                                   float64
Low                                    float64
Close                                  float64
Volume                                 float64
MACD_line                              float64
Signal_line                            float64
Zero_line                                int64
Signal                                   int64
dtype: object

In [59]:
import yfinance as yf
signals = yf.download('AAPL',start='2015-01-01')

[*********************100%***********************]  1 of 1 completed


In [60]:
signals

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.565699,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.873640,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.875895,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.210682,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.140911,237458000
...,...,...,...,...,...,...
2023-03-23,158.830002,161.550003,157.679993,158.929993,158.929993,67622100
2023-03-24,158.860001,160.339996,157.850006,160.250000,160.250000,59196500
2023-03-27,159.940002,160.770004,157.869995,158.279999,158.279999,52390300


In [61]:

cerebro = bt.Cerebro()
feed = bt.feeds.PandasData(dataname=signals)
cerebro.adddata(feed)
#cerebro.addstrategy(MyStrategy)
cerebro.run()

In [63]:
cerebro.plot()



ImportError: cannot import name 'warnings' from 'matplotlib.dates' (c:\Users\jdlugosz\AppData\Local\miniconda3\lib\site-packages\matplotlib\dates.py)